In [1]:
pip install pandas numpy scikit-learn matplotlib tensorflow


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install keras-tuner


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install bertopic[all]


Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install tf-keras

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install tqdm sentence-transformers torch transformers

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install statsmodels


Note: you may need to restart the kernel to use updated packages.


GRU

In [ ]:
#KALAU MAU PAKAI GPU
#import tensorflow as tf
#gpus = tf.config.list_physical_devices('GPU')
#if gpus:
#    print("✅ GPU tersedia:", gpus)
#    try:
#        tf.config.experimental.set_memory_growth(gpus[0], True)
#    except RuntimeError as e:
#        print(e)
#else:
#    print("⚠️ GPU tidak tersedia, menggunakan CPU")

import os
os.environ["LOKY_MAX_CPU_COUNT"] = "2"

import pandas as pd
import numpy as np
import time
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, Dropout, concatenate, TimeDistributed, BatchNormalization, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from tqdm import tqdm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

PRICE_FEATURES = ['open', 'high', 'low', 'close', 'volume']
aapl = pd.read_csv("AAPL_2020_2025.csv")
apple_df = pd.read_csv("apple_with_sentiment.csv")

for df in [aapl, apple_df]:
    df['date'] = pd.to_datetime(df['date']).dt.tz_localize(None)

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

def run_topic_pipeline(news_df):
    embeddings = embedding_model.encode(news_df['content'].tolist(), show_progress_bar=True)
    topic_model = BERTopic(
        embedding_model=embedding_model,
        calculate_probabilities=True,
        min_topic_size=15,
        low_memory=True,
        verbose=True
    )
    topics, probs = topic_model.fit_transform(news_df['content'], embeddings)
    news_df['topic_prob'] = [np.array(p) for p in probs]

    num_topics = len(set(topics)) - (1 if -1 in topics else 0)
    daily_topic_sent = {}
    for date, group in news_df.groupby('date'):
        topic_sent = np.zeros(num_topics)
        topic_weight = np.zeros(num_topics)
        for _, row in group.iterrows():
            for i in range(num_topics):
                topic_sent[i] += row['sentiment_score'] * row['topic_prob'][i]
                topic_weight[i] += row['topic_prob'][i]
        avg_sent = [topic_sent[i]/topic_weight[i] if topic_weight[i]!=0 else 0 for i in range(num_topics)]
        daily_topic_sent[date] = avg_sent

    sent_df = pd.DataFrame.from_dict(daily_topic_sent, orient='index')
    sent_df.index = pd.to_datetime(sent_df.index).tz_localize(None).normalize()
    return sent_df

topic_aapl = run_topic_pipeline(apple_df)

def merge_price_topic(stock_df, topic_df):
    df = stock_df.copy()
    df = df[df['date'] >= '2021-01-01']
    df.set_index('date', inplace=True)
    return pd.merge(df, topic_df, left_index=True, right_index=True, how='inner').dropna()

merged_aapl = merge_price_topic(aapl, topic_aapl)
WINDOW_SIZE = 360

def make_dataset(df, window_size):
    X_price, X_topic, y, dates = [], [], [], []
    scaler = StandardScaler()
    scaled_prices = scaler.fit_transform(df[PRICE_FEATURES])
    for i in range(window_size, len(df) - 1):
        price = scaled_prices[i-window_size:i]
        topic = df.iloc[i-window_size:i, len(PRICE_FEATURES):].to_numpy(dtype=np.float32)
        target = df.iloc[i+1]['close']
        date = df.index[i+1]
        X_price.append(price)
        X_topic.append(topic)
        y.append(target)
        dates.append(date)
    return (
        np.array(X_price, dtype=np.float32),
        np.array(X_topic, dtype=np.float32),
        np.array(y, dtype=np.float32),
        dates
    )

Xp, Xt, y, y_dates = make_dataset(merged_aapl, WINDOW_SIZE)

def build_gru_model(Xp, Xt):
    in_price = Input(shape=(Xp.shape[1], Xp.shape[2]))
    x1 = GRU(64, return_sequences=True)(in_price)
    x1 = Dropout(0.3)(x1)
    x1 = GRU(16)(x1)
    x1 = Dropout(0.3)(x1)

    in_topic = Input(shape=(Xt.shape[1], Xt.shape[2]))
    x2 = TimeDistributed(Dense(64, activation='relu'))(in_topic)
    x2 = BatchNormalization()(x2)
    x2 = Dropout(0.3)(x2)
    x2 = TimeDistributed(Dense(32, activation='relu'))(x2)
    x2 = BatchNormalization()(x2)
    x2 = Dropout(0.3)(x2)
    x2 = GlobalAveragePooling1D()(x2)

    x = concatenate([x1, x2])
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(32, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    out = Dense(1, activation='linear')(x)

    model = Model(inputs=[in_price, in_topic], outputs=out)
    model.compile(optimizer=Adam(learning_rate=1e-3), loss='mse')
    return model

def walk_forward_gru(Xp, Xt, y, dates, start=0.8):
    n_total = len(y)
    n_train = int(n_total * start)
    y_pred, y_true = [], []
    start_all = time.time()
    for i in tqdm(range(n_train, n_total), desc="Walk-Forward GRU"):
        t0 = time.time()
        model = build_gru_model(Xp, Xt)
        model.fit([Xp[:i], Xt[:i]], y[:i], epochs=15, batch_size=32, verbose=0)
        pred = model.predict([Xp[i:i+1], Xt[i:i+1]], verbose=0)[0][0]
        y_pred.append(pred)
        y_true.append(y[i])
        t1 = time.time()
        print(f"Step {i}/{n_total} - Time: {t1 - t0:.2f}s")
    print(f"Total time: {(time.time() - start_all)/60:.2f} minutes")
    return y_true, y_pred

def evaluate_baseline(y_true, y_pred, label="GRU"):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    print(f"📊 {label} — MAE: {mae:.2f}, RMSE: {rmse:.2f}, R²: {r2:.4f}")
    return mae, rmse, r2

def baseline_random_walk(y_true):
    return y_true[:-1], y_true[1:]

def baseline_arima(df_close):
    history = list(df_close[:WINDOW_SIZE])
    predictions = []
    for t in range(WINDOW_SIZE, len(df_close)-1):
        model = ARIMA(history, order=(5,1,0))
        model_fit = model.fit()
        output = model_fit.forecast()
        predictions.append(output[0])
        history.append(df_close[t])
    return df_close[WINDOW_SIZE+1:], predictions[1:]

def baseline_sarima(df_close):
    history = list(df_close[:WINDOW_SIZE])
    predictions = []
    for t in range(WINDOW_SIZE, len(df_close)-1):
        model = SARIMAX(history, order=(1,1,1), seasonal_order=(1,1,1,12))
        model_fit = model.fit(disp=False)
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(df_close[t])
    return df_close[WINDOW_SIZE+1:], predictions[1:]

print("\n🔁 Training GRU with Walk-Forward Validation + ETA...")
y_true_gru, y_pred_gru = walk_forward_gru(Xp, Xt, y, y_dates)
evaluate_baseline(y_true_gru, y_pred_gru, "GRU")

print("\n📉 Evaluating Baseline Models...")
y_rw_true, y_rw_pred = baseline_random_walk(list(y))
evaluate_baseline(y_rw_true, y_rw_pred, "Random Walk")

y_arima_true, y_arima_pred = baseline_arima(list(merged_aapl['close'].values))
evaluate_baseline(y_arima_true, y_arima_pred, "ARIMA")

y_sarima_true, y_sarima_pred = baseline_sarima(list(merged_aapl['close'].values))
evaluate_baseline(y_sarima_true, y_sarima_pred, "SARIMA")

plt.figure(figsize=(12, 4))
plt.plot(y_true_gru[:100], label="Actual", marker='o')
plt.plot(y_pred_gru[:100], label="GRU Predicted", marker='x')
plt.title("AAPL - GRU Walk-Forward Prediction (First 100 Days)")
plt.legend(); plt.grid(); plt.show()



# 💾 Simpan prediksi untuk uji statistik
np.save("y_true.npy", np.array(y_true_gru))
np.save("y_pred_gru.npy", np.array(y_pred_gru))

KeyboardInterrupt: 

Versi coba tanpa Retracing / Loopnya di luar

In [ ]:
import os
os.environ["LOKY_MAX_CPU_COUNT"] = "2"
#KALAU MAU PAKAI GPU
#import tensorflow as tf
#gpus = tf.config.list_physical_devices('GPU')
#if gpus:
#    print("✅ GPU tersedia:", gpus)
#    try:
#        tf.config.experimental.set_memory_growth(gpus[0], True)
#    except RuntimeError as e:
#        print(e)
#else:
#    print("⚠️ GPU tidak tersedia, menggunakan CPU")

import pandas as pd
import numpy as np
import time
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, Dropout, concatenate, TimeDistributed, BatchNormalization, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from tqdm import tqdm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

PRICE_FEATURES = ['open', 'high', 'low', 'close', 'volume']
aapl = pd.read_csv("AAPL_2020_2025.csv")
apple_df = pd.read_csv("apple_with_sentiment.csv")

for df in [aapl, apple_df]:
    df['date'] = pd.to_datetime(df['date']).dt.tz_localize(None)

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

def run_topic_pipeline(news_df):
    embeddings = embedding_model.encode(news_df['content'].tolist(), show_progress_bar=True)
    topic_model = BERTopic(
        embedding_model=embedding_model,
        calculate_probabilities=True,
        min_topic_size=15,
        low_memory=True,
        verbose=True
    )
    topics, probs = topic_model.fit_transform(news_df['content'], embeddings)
    news_df['topic_prob'] = [np.array(p) for p in probs]

    num_topics = len(set(topics)) - (1 if -1 in topics else 0)
    daily_topic_sent = {}
    for date, group in news_df.groupby('date'):
        topic_sent = np.zeros(num_topics)
        topic_weight = np.zeros(num_topics)
        for _, row in group.iterrows():
            for i in range(num_topics):
                topic_sent[i] += row['sentiment_score'] * row['topic_prob'][i]
                topic_weight[i] += row['topic_prob'][i]
        avg_sent = [topic_sent[i]/topic_weight[i] if topic_weight[i]!=0 else 0 for i in range(num_topics)]
        daily_topic_sent[date] = avg_sent

    sent_df = pd.DataFrame.from_dict(daily_topic_sent, orient='index')
    sent_df.index = pd.to_datetime(sent_df.index).tz_localize(None).normalize()
    return sent_df

topic_aapl = run_topic_pipeline(apple_df)

def merge_price_topic(stock_df, topic_df):
    df = stock_df.copy()
    df = df[df['date'] >= '2021-01-01']
    df.set_index('date', inplace=True)
    return pd.merge(df, topic_df, left_index=True, right_index=True, how='inner').dropna()

merged_aapl = merge_price_topic(aapl, topic_aapl)
WINDOW_SIZE = 360

def make_dataset(df, window_size):
    X_price, X_topic, y, dates = [], [], [], []
    scaler = StandardScaler()
    scaled_prices = scaler.fit_transform(df[PRICE_FEATURES])
    for i in range(window_size, len(df) - 1):
        price = scaled_prices[i-window_size:i]
        topic = df.iloc[i-window_size:i, len(PRICE_FEATURES):].to_numpy(dtype=np.float32)
        target = df.iloc[i+1]['close']
        date = df.index[i+1]
        X_price.append(price)
        X_topic.append(topic)
        y.append(target)
        dates.append(date)
    return (
        np.array(X_price, dtype=np.float32),
        np.array(X_topic, dtype=np.float32),
        np.array(y, dtype=np.float32),
        dates
    )

Xp, Xt, y, y_dates = make_dataset(merged_aapl, WINDOW_SIZE)

def build_gru_model(Xp, Xt):
    in_price = Input(shape=(Xp.shape[1], Xp.shape[2]))
    x1 = GRU(64, return_sequences=True)(in_price)
    x1 = Dropout(0.3)(x1)
    x1 = GRU(16)(x1)
    x1 = Dropout(0.3)(x1)

    in_topic = Input(shape=(Xt.shape[1], Xt.shape[2]))
    x2 = TimeDistributed(Dense(64, activation='relu'))(in_topic)
    x2 = BatchNormalization()(x2)
    x2 = Dropout(0.3)(x2)
    x2 = TimeDistributed(Dense(32, activation='relu'))(x2)
    x2 = BatchNormalization()(x2)
    x2 = Dropout(0.3)(x2)
    x2 = GlobalAveragePooling1D()(x2)

    x = concatenate([x1, x2])
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(32, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    out = Dense(1, activation='linear')(x)

    model = Model(inputs=[in_price, in_topic], outputs=out)
    model.compile(optimizer=Adam(learning_rate=1e-3), loss='mse')
    return model

def walk_forward_gru_no_retrace(Xp, Xt, y, dates, start=0.8, epochs=15):
    n_total = len(y)
    n_train = int(n_total * start)
    y_pred, y_true = [], []
    start_all = time.time()
    model = build_gru_model(Xp, Xt)
    for i in tqdm(range(n_train, n_total), desc="Walk-Forward GRU (No Retrace)"):
        t0 = time.time()
        model.fit([Xp[:i], Xt[:i]], y[:i], epochs=epochs, batch_size=32, verbose=0)
        pred_fn = model.predict
        pred = pred_fn([Xp[i:i+1], Xt[i:i+1]], verbose=0)[0][0]
        y_pred.append(pred)
        y_true.append(y[i])
        print(f"Step {i}/{n_total} - Time: {time.time() - t0:.2f}s")
    print(f"Total time: {(time.time() - start_all)/60:.2f} minutes")
    return y_true, y_pred

def evaluate_baseline(y_true, y_pred, label="GRU"):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    print(f"📊 {label} — MAE: {mae:.2f}, RMSE: {rmse:.2f}, R²: {r2:.4f}")
    return mae, rmse, r2

def baseline_random_walk(y_true):
    return y_true[:-1], y_true[1:]

def baseline_arima(df_close):
    history = list(df_close[:WINDOW_SIZE])
    predictions = []
    for t in range(WINDOW_SIZE, len(df_close)-1):
        model = ARIMA(history, order=(5,1,0))
        model_fit = model.fit()
        output = model_fit.forecast()
        predictions.append(output[0])
        history.append(df_close[t])
    return df_close[WINDOW_SIZE+1:], predictions[1:]

def baseline_sarima(df_close):
    history = list(df_close[:WINDOW_SIZE])
    predictions = []
    for t in range(WINDOW_SIZE, len(df_close)-1):
        model = SARIMAX(history, order=(1,1,1), seasonal_order=(1,1,1,12))
        model_fit = model.fit(disp=False)
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(df_close[t])
    return df_close[WINDOW_SIZE+1:], predictions[1:]

print("\n🔁 Training GRU with Walk-Forward Validation + ETA...")
y_true_gru, y_pred_gru = walk_forward_gru_no_retrace(Xp, Xt, y, y_dates)
evaluate_baseline(y_true_gru, y_pred_gru, "GRU")

print("\n📉 Evaluating Baseline Models...")
y_rw_true, y_rw_pred = baseline_random_walk(list(y))
evaluate_baseline(y_rw_true, y_rw_pred, "Random Walk")

y_arima_true, y_arima_pred = baseline_arima(list(merged_aapl['close'].values))
evaluate_baseline(y_arima_true, y_arima_pred, "ARIMA")

y_sarima_true, y_sarima_pred = baseline_sarima(list(merged_aapl['close'].values))
evaluate_baseline(y_sarima_true, y_sarima_pred, "SARIMA")

plt.figure(figsize=(12, 4))
plt.plot(y_true_gru[:100], label="Actual", marker='o')
plt.plot(y_pred_gru[:100], label="GRU Predicted", marker='x')
plt.title("AAPL - GRU Walk-Forward Prediction (First 100 Days)")
plt.legend(); plt.grid(); plt.show()

np.save("y_pred_gru.npy", np.array(y_pred_gru))


Versi Pakai Expanding Windowing

In [ ]:
# GRU with expanding-window validation (step=5) + baselines + metrics + plot
#KALAU MAU PAKAI GPU
#import tensorflow as tf
#gpus = tf.config.list_physical_devices('GPU')
#if gpus:
#    print("✅ GPU tersedia:", gpus)
#    try:
#        tf.config.experimental.set_memory_growth(gpus[0], True)
#    except RuntimeError as e:
#        print(e)
#else:
#    print("⚠️ GPU tidak tersedia, menggunakan CPU")

import os
os.environ["LOKY_MAX_CPU_COUNT"] = "4"

import pandas as pd
import numpy as np
import time
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, Dropout, concatenate, TimeDistributed, BatchNormalization, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from tqdm import tqdm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# ======= Data Load =======
PRICE_FEATURES = ['open', 'high', 'low', 'close', 'volume']
aapl = pd.read_csv("AAPL_2020_2025.csv")
apple_df = pd.read_csv("apple_with_sentiment.csv")
for df in [aapl, apple_df]:
    df['date'] = pd.to_datetime(df['date']).dt.tz_localize(None)

# ======= Topic Sentiment Pipeline =======
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
def run_topic_pipeline(news_df):
    embeddings = embedding_model.encode(news_df['content'].tolist(), show_progress_bar=True)
    topic_model = BERTopic(
        embedding_model=embedding_model,
        calculate_probabilities=True,
        min_topic_size=15,
        low_memory=True,
        verbose=True
    )
    topics, probs = topic_model.fit_transform(news_df['content'], embeddings)
    news_df['topic_prob'] = [np.array(p) for p in probs]
    num_topics = len(set(topics)) - (1 if -1 in topics else 0)
    daily_topic_sent = {}
    for date, group in news_df.groupby('date'):
        topic_sent = np.zeros(num_topics)
        topic_weight = np.zeros(num_topics)
        for _, row in group.iterrows():
            for i in range(num_topics):
                topic_sent[i] += row['sentiment_score'] * row['topic_prob'][i]
                topic_weight[i] += row['topic_prob'][i]
        avg_sent = [topic_sent[i]/topic_weight[i] if topic_weight[i]!=0 else 0 for i in range(num_topics)]
        daily_topic_sent[date] = avg_sent
    sent_df = pd.DataFrame.from_dict(daily_topic_sent, orient='index')
    sent_df.index = pd.to_datetime(sent_df.index).tz_localize(None).normalize()
    return sent_df

topic_aapl = run_topic_pipeline(apple_df)

# ======= Merge & Prepare Dataset =======
def merge_price_topic(stock_df, topic_df):
    df = stock_df.copy()
    df = df[df['date'] >= '2021-01-01']
    df.set_index('date', inplace=True)
    return pd.merge(df, topic_df, left_index=True, right_index=True, how='inner').dropna()

merged_aapl = merge_price_topic(aapl, topic_aapl)
WINDOW_SIZE = 360

def make_dataset(df, window_size):
    X_price, X_topic, y, dates = [], [], [], []
    scaler = StandardScaler()
    scaled_prices = scaler.fit_transform(df[PRICE_FEATURES])
    for i in range(window_size, len(df) - 1):
        price = scaled_prices[i-window_size:i]
        topic = df.iloc[i-window_size:i, len(PRICE_FEATURES):].to_numpy(dtype=np.float32)
        target = df.iloc[i+1]['close']
        date = df.index[i+1]
        X_price.append(price)
        X_topic.append(topic)
        y.append(target)
        dates.append(date)
    return (
        np.array(X_price, dtype=np.float32),
        np.array(X_topic, dtype=np.float32),
        np.array(y, dtype=np.float32),
        dates
    )

Xp, Xt, y, y_dates = make_dataset(merged_aapl, WINDOW_SIZE)

# ======= GRU Model =======
def build_gru_model(Xp, Xt):
    in_price = Input(shape=(Xp.shape[1], Xp.shape[2]))
    x1 = GRU(64, return_sequences=True)(in_price)
    x1 = Dropout(0.3)(x1)
    x1 = GRU(16)(x1)
    x1 = Dropout(0.3)(x1)

    in_topic = Input(shape=(Xt.shape[1], Xt.shape[2]))
    x2 = TimeDistributed(Dense(64, activation='relu'))(in_topic)
    x2 = BatchNormalization()(x2)
    x2 = Dropout(0.3)(x2)
    x2 = TimeDistributed(Dense(32, activation='relu'))(x2)
    x2 = BatchNormalization()(x2)
    x2 = Dropout(0.3)(x2)
    x2 = GlobalAveragePooling1D()(x2)

    x = concatenate([x1, x2])
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(32, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    out = Dense(1, activation='linear')(x)

    model = Model(inputs=[in_price, in_topic], outputs=out)
    model.compile(optimizer=Adam(learning_rate=1e-3), loss='mse')
    return model

# ======= Expanding Step=5 =======
def walk_forward_gru_expanding_step5(Xp, Xt, y, dates, start=0.8, epochs=10):
    n_total = len(y)
    n_train = int(n_total * start)
    y_pred, y_true = [], []
    model = build_gru_model(Xp, Xt)
    step_durations = []

    for i in tqdm(range(n_train, n_total, 5), desc="Expanding Step=5"):
        start_time = time.time()
        model.fit([Xp[:i], Xt[:i]], y[:i], epochs=epochs, batch_size=32, verbose=0)
        for j in range(i, min(i+5, n_total)):
            pred = model.predict([Xp[j:j+1], Xt[j:j+1]], verbose=0)[0][0]
            y_pred.append(pred)
            y_true.append(y[j])
        step_time = time.time() - start_time
        eta = np.mean(step_durations) * ((n_total - i - 1) // 5) / 60 if step_durations else 0
        step_durations.append(step_time)
        print(f"Step {i}-{min(i+5, n_total)-1} done in {step_time:.2f}s | ETA: {eta:.2f} min")

    return y_true, y_pred

# ======= Evaluation =======
def evaluate_baseline(y_true, y_pred, label="Model"):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    print(f"📊 {label} — MAE: {mae:.2f}, RMSE: {rmse:.2f}, R²: {r2:.4f}")
    return mae, rmse, r2

def baseline_random_walk(y_true):
    return y_true[:-1], y_true[1:]

def baseline_arima(df_close):
    history = list(df_close[:WINDOW_SIZE])
    predictions = []
    for t in range(WINDOW_SIZE, len(df_close)-1):
        model = ARIMA(history, order=(5,1,0))
        model_fit = model.fit()
        output = model_fit.forecast()
        predictions.append(output[0])
        history.append(df_close[t])
    return df_close[WINDOW_SIZE+1:], predictions[1:]

def baseline_sarima(df_close):
    history = list(df_close[:WINDOW_SIZE])
    predictions = []
    for t in range(WINDOW_SIZE, len(df_close)-1):
        model = SARIMAX(history, order=(1,1,1), seasonal_order=(1,1,1,12))
        model_fit = model.fit(disp=False)
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(df_close[t])
    return df_close[WINDOW_SIZE+1:], predictions[1:]

# ======= Run All =======
print("\n🔁 Training GRU with Expanding Step=5...")
y_true_gru, y_pred_gru = walk_forward_gru_expanding_step5(Xp, Xt, y, y_dates)

np.save("y_true_gru_step5.npy", np.array(y_true_gru))
np.save("y_pred_gru_step5.npy", np.array(y_pred_gru))

evaluate_baseline(y_true_gru, y_pred_gru, "GRU (Step=5)")

print("\n📉 Evaluating Baseline Models...")
y_rw_true, y_rw_pred = baseline_random_walk(list(y))
evaluate_baseline(y_rw_true, y_rw_pred, "Random Walk")

y_arima_true, y_arima_pred = baseline_arima(list(merged_aapl['close'].values))
evaluate_baseline(y_arima_true, y_arima_pred, "ARIMA")

y_sarima_true, y_sarima_pred = baseline_sarima(list(merged_aapl['close'].values))
evaluate_baseline(y_sarima_true, y_sarima_pred, "SARIMA")

plt.figure(figsize=(12, 4))
plt.plot(y_true_gru[:100], label="Actual", marker='o')
plt.plot(y_pred_gru[:100], label="GRU Predicted", marker='x')
plt.title("AAPL - GRU Expanding Step=5 Prediction (First 100 Days)")
plt.legend(); plt.grid(); plt.show()
